In [4]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

Cuda is not available!
logicqubit version 1.5.8


https://arxiv.org/pdf/1907.09386.pdf

In [5]:
def commutator(A, B):
    C = A*B-B*A
    return np.all(C.get() == 0)

In [6]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [7]:
III = ID.kron(ID).kron(ID)
XXX = X.kron(X).kron(X)
XIZ = X.kron(ID).kron(Z)
XYI = X.kron(Y).kron(ID)
XIZ = X.kron(ID).kron(Z)
YYY = Y.kron(Y).kron(Y)
ZZZ = Z.kron(Z).kron(Z)
IZZ = ID.kron(Z).kron(Z)
ZZI = ID.kron(Z).kron(ID)

In [8]:
commutator(XYI, XIZ)

True

In [9]:
# autovalores de X,Y,Z: [1, -1] e de I: [1, 1]
sig_iss = np.kron(np.kron([1, 1], [1, -1]), [1, -1])
sig_ssi = np.kron(np.kron([1, -1], [1, -1]), [1, 1])
sig_sis = np.kron(np.kron([1, -1], [1, 1]), [1, -1])
sig_sis

array([ 1, -1,  1, -1, -1,  1, -1,  1])

In [10]:
H = III + XYI*3 + XIZ*10

min(scipy.linalg.eig(H.get())[0])

(-12+4.1241847957050005e-32j)

In [11]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_3q(q1, q2, q3, params):
    q1.RY(params[0])
    q2.RY(params[1])
    q3.RY(params[2])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RX(params[3])
    q2.RX(params[4])
    q3.RX(params[5])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RY(params[6])
    q2.RY(params[7])
    q3.RY(params[8])
    q2.CNOT(q1)
    q3.CNOT(q2)
    q1.RY(params[9])
    q2.RY(params[10])
    q3.RY(params[11])
    q2.CNOT(q1)
    q3.CNOT(q2)

In [12]:
def expectation_3q(params):
    logicQuBit  = LogicQuBit(3)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()

    ansatz_3q(q1,q2,q3,params)
    #ansatz([q1,q2,q3],params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_3q, [0,0,0,0,0,0,0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:586: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[k] = func(sim[k])
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:611: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxe
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:618: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:614: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxr
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:637: ComplexWarning: Casting complex values to real discards the imaginary part
  fsim[-1] = fxcc
/home/cleoner/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:627: ComplexWarning: Casting com

 final_simplex: (array([[-1.70536841e+00, -1.00539050e-01,  1.55861477e-01,
         1.43285592e+00,  7.21512712e-01,  4.20845303e-02,
         1.09009478e-01,  3.22594690e-02, -1.58316966e-01,
        -4.84445090e-01,  2.47706805e-02, -7.58020841e-01],
       [-1.72932470e+00, -1.11875984e-01,  1.48727714e-01,
         1.45605885e+00,  7.19279604e-01,  4.18493832e-03,
         1.38757640e-01,  1.35012392e-02, -1.66798557e-01,
        -4.85511796e-01,  4.19130555e-02, -7.24793089e-01],
       [-1.71587546e+00, -1.16578201e-01,  1.31684364e-01,
         1.43959437e+00,  6.84974479e-01,  2.05536298e-02,
         1.07468660e-01,  5.54096191e-02, -1.57343206e-01,
        -4.69880836e-01,  2.61660302e-02, -7.27302825e-01],
       [-1.68926830e+00, -1.28022390e-01,  1.27347166e-01,
         1.44220497e+00,  6.59737147e-01,  2.54488244e-03,
         1.23687602e-01,  4.34666389e-02, -1.59325585e-01,
        -4.65406496e-01,  2.60530944e-02, -7.03618705e-01],
       [-1.70178393e+00, -1.3903982

In [15]:
def expectation_value(measurements, base = np.array([1,-1,-1,1,-1,1,1,-1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xxx(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    q3.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

# [xyi, xiz]=0 precisamos de apenas uma medida no eixo XYZ
def sigma_xyz(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em XYZ
    q1.RY(-pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    xyi = expectation_value(result, sig_ssi)
    xiz = expectation_value(result, sig_sis) 
    return xyi, xiz

def sigma_yyy(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    q3.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2,q3])
    result = expectation_value(result)
    return result

def sigma_zzz(params):
    logicQuBit  = LogicQuBit(3, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    q3 = Qubit()
    
    ansatz_3q(q1,q2,q3,params)
          
    result = logicQuBit.Measure([q1,q2,q3])
    zzz = expectation_value(result)
    izz = expectation_value(result, sig_iss) # [zzz, izz] = 0
    return zzz, izz 

def expectation_energy(params):
    xyi, xiz =  sigma_xyz(params)

    result = 1 + 3*xyi + 10*xiz
    return result

In [16]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0,0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[-1.59861417,  0.67851413,  0.10077095, -0.92422256, -0.11976661,
         0.00917929,  1.13129902,  0.47232269, -0.86804904,  0.11303789,
         0.21045843, -0.08121447],
       [-1.59895826,  0.67857134,  0.10084076, -0.92411586, -0.11940067,
         0.00888377,  1.1313394 ,  0.4721589 , -0.86791851,  0.11316486,
         0.21029665, -0.08118751],
       [-1.59883373,  0.67864737,  0.10089682, -0.9241835 , -0.11958172,
         0.00921565,  1.13123184,  0.47222441, -0.86796823,  0.11303257,
         0.21034507, -0.08105163],
       [-1.59863681,  0.67853123,  0.10099701, -0.92431596, -0.11956898,
         0.00894363,  1.13132499,  0.47220322, -0.8681013 ,  0.11301176,
         0.21051405, -0.0810089 ],
       [-1.59891672,  0.67803267,  0.10070373, -0.92361073, -0.11958812,
         0.00895073,  1.13074031,  0.47220744, -0.86746476,  0.1132327 ,
         0.21086732, -0.08111697],
       [-1.59895044,  0.67867105,  0.10066558, -0.9243782 , -0.11958008,
     

In [17]:
def gradient(params, evaluate, shift=pi/2):
    n_params = params.shape[0]
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [24]:
params = np.random.uniform(-np.pi, np.pi, 12)
last_params = np.zeros(12)

In [25]:
lr = 0.1
err = 1
while err > 1e-5:
    grad = gradient(params, expectation_energy, pi/1.1)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_11017/3485820173.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


0.8184556076668226
0.38752317161238425
0.31702405800521294
0.2114378341002251
0.12622217096968308
0.06988522765929
0.036757536629974275
0.017942924481501726
0.0070756162271398415
0.0005119468375004432
0.003698812536389179
0.006590165046403695
0.008719561745389709
0.010398725049715174
0.01181080663570816
0.01306999962503802
0.014252341439315447
0.015412060931818933
0.01659058414055703
0.01782174325629804
0.019134978012696813
0.020557450358407475
0.022115558994343143
0.02383611987328671
0.02574736439465522
0.02787984548498948
0.03026730594273791
0.032947538679256816
0.03596324558849784
0.03936287309745537
0.04320135940688802
0.04754065935909127
0.05244980096077244
0.058004049267572466
0.06428247867973151
0.0713628543250511
0.07931219144133625
0.08817077437715488
0.09792705008058987
0.1084813340462448
0.11959893426643273
0.1308598055205104
0.1416230660430793
0.1510374797164561
0.1581322214338978
0.1619990907937687
0.16202305509789933
0.15806412646508625
0.15049679275216576
0.1400902031428

In [26]:
expectation_energy(params)

(-11.99999999921216+0j)